# Program für Roboter- und Computervision

### Bekomme den Stream der Kamera und verwandle es in ein grau Stufen Bild
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html?highlight=video

In [2]:
import cv2
import numpy as np 
from matplotlib import pyplot as plt

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html?highlight=imshow
    #canny Edge Detection
    edges = cv2.Canny(gray,100,200)
    # 1 Fenster mit Graustufe
    namedWindow1 = "gray"
    cv2.namedWindow(namedWindow1)
    cv2.moveWindow(namedWindow1,0,0)
    cv2.imshow(namedWindow1, gray)
    # 2 Fenster mit Kantenbild
    namedWindow2 = "edges"
    cv2.namedWindow(namedWindow2)
    cv2.moveWindow(namedWindow2,640,0)
    cv2.imshow(namedWindow2,edges)
    # 3 Fenster 
    namedWindow3 = "weiteres"
    cv2.namedWindow(namedWindow3)
    cv2.moveWindow(namedWindow3,1280,0)
    cv2.imshow(namedWindow3,edges)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
